<a href="https://colab.research.google.com/github/Loirinhm/news_classification/blob/main/news_classifier_gpu_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# News Classifier

In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

In [2]:
print(tf.__version__)

2.16.1


## Preparing the Data

In [3]:
# load as panda series
data_texts = pd.read_csv('data_texts.csv')
data_labels = pd.read_csv('data_labels.csv')

In [4]:
# split the data into training (train_ds) and testing (test_ds) for the bert model
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(data_texts, data_labels, test_size=0.2, random_state=29)

In [5]:
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(data_labels.values.ravel())  # Use values to get a numpy array, then ravel() to convert to 1D array

train_labels_t = label_encoder.transform(train_labels.values.ravel())  # Use values to get a numpy array, then ravel() to convert to 1D array
val_labels_t = label_encoder.transform(val_labels.values.ravel())  # Use values to get a numpy array, then ravel() to convert to 1D array

In [6]:
y_train = to_categorical(train_labels_t, num_classes=29)
y_val = to_categorical(val_labels_t, num_classes=29)

In [7]:
batch_size = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = (
    tf.data.Dataset
        .from_tensor_slices((train_texts, y_train))
        .repeat()
        .shuffle(2048)
        .batch(batch_size)
        .prefetch(buffer_size=AUTOTUNE)
)

val_ds = (
    tf.data.Dataset
        .from_tensor_slices((val_texts, y_val))
        .batch(batch_size)
        .prefetch(buffer_size=AUTOTUNE)
)

## Building, Training, and Evaluating Models

### BERT

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
!pip install tensorflow-text

In [ ]:
!pip install --upgrade tensorflow-hub

In [8]:
!pip show tensorflow
!pip show keras
!pip show tensorflow-text
!pip show tensorflow-hub

Name: tensorflow
Version: 2.16.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine-rl, tensorflow-text, tf_keras
Name: keras
Version: 3.2.1
Summary: Multi-backend Keras.
Home-page: https://github.com/keras-team/keras
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, h5py, ml-dtypes, namex, numpy, optree, rich
Required-by: tensorflow
Name: tensorflow-text
Version: 2.16.1
Summary: TF.Text is a TensorFlow library of text

In [9]:
import tensorflow_hub as hub

AttributeError: module 'tensorflow._api.v2.compat.v2.__internal__' has no attribute 'register_load_context_function'

In [ ]:
import tensorflow_text as text

In [13]:
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

ModuleNotFoundError: No module named 'keras.api'

In [20]:
def build_model(tfhub_handle_preprocess, tfhub_handle_encoder, dropout_rate=0.1, num_classes=29, epochs=10, batch_size=32):
    text_input = Input(shape=(), dtype=tf.string, name='text_input')
    preprocessor = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessor')
    encoder_inputs = preprocessor(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    pooled_output = outputs["pooled_output"]
    sequence_output = outputs["sequence_output"]

    dropout = Dropout(dropout_rate)(pooled_output)
    output = Dense(num_classes, activation='softmax', name='output')(dropout)

    model = Model(inputs=text_input, outputs=output)

    loss = CategoricalCrossentropy()
    metrics = tf.metrics.CategoricalAccuracy()

    steps_per_epoch = len(train_texts) // batch_size
    num_train_steps = steps_per_epoch * epochs

    init_lr = 3e-5

    lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
        initial_learning_rate=init_lr,
        decay_steps=num_train_steps,
        end_learning_rate=0.0
    )

    optimizer = adamw_experimental.Adam(learning_rate=lr_schedule)

    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

    return model

In [17]:
tfhub_handle_preprocess = 'https://kaggle.com/models/tensorflow/albert/TensorFlow2/en-preprocess/3'
tfhub_handle_encoder = 'https://www.kaggle.com/models/tensorflow/bert/TensorFlow2/bert-en-uncased-l-10-h-512-a-8/2'

In [21]:
model_A = build_model(tfhub_handle_preprocess, tfhub_handle_encoder)

RuntimeError: Op type not registered 'NormalizeUTF8' in binary running on 01ea599d0f67. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib (e.g. `tf.contrib.resampler`), accessing should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.